<a href="https://colab.research.google.com/github/Iddhie/BlockchainDataset/blob/main/exampledf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime
import requests
import pandas as pd
import matplotlib.pyplot as plt

ether_api_key='FVIGP2ZZX9I44Q1NTPW9BC1R2N5D8IPZWH'
ether_url='https://api.etherscan.io/api'
subgraph_url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'

In [ ]:
query = '''
{
  swaps(first: 1000, orderBy: timestamp, orderDirection: desc) {
    timestamp
    origin
    amountUSD
    token0 {
      id
      symbol
      feesUSD
      totalValueLockedUSD
      volumeUSD
    }
    amount0
    token1 {
      id
      symbol
      feesUSD
      totalValueLockedUSD
      volumeUSD

    }
    amount1
    transaction {
      gasPrice
    }
  }
}
'''
headers = {
    'Content-Type': 'application/json',
}

# Make a POST request to the Uniswap v3 subgraph endpoint
response = requests.post(subgraph_url, json={'query': query}, headers=headers)

if response.status_code == 200:
    data = response.json()
    swaps_data = data['data']['swaps']  # Extract 'swaps' data from the response

    # Convert the data to a Pandas DataFrame
    df = pd.json_normalize(swaps_data)
    df = pd.DataFrame(df)
else:
    print(f"Request failed with status code {response.status_code}")
    print(response.text)  # Print the error message if the request fails

# Convert 'amount0', 'amount1', and 'amountUSD' columns to numeric and round to 4 decimal places
df['amount0'] = pd.to_numeric(df['amount0'], errors='coerce').round(4)
df['amount1'] = pd.to_numeric(df['amount1'], errors='coerce').round(4)
df['amountUSD'] = pd.to_numeric(df['amountUSD'], errors='coerce').round(4)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   timestamp                   1000 non-null   object 
 1   origin                      1000 non-null   object 
 2   amountUSD                   1000 non-null   float64
 3   amount0                     1000 non-null   float64
 4   amount1                     1000 non-null   float64
 5   token0.id                   1000 non-null   object 
 6   token0.symbol               1000 non-null   object 
 7   token0.feesUSD              1000 non-null   object 
 8   token0.totalValueLockedUSD  1000 non-null   object 
 9   token0.volumeUSD            1000 non-null   object 
 10  token1.id                   1000 non-null   object 
 11  token1.symbol               1000 non-null   object 
 12  token1.feesUSD              1000 non-null   object 
 13  token1.totalValueLockedUSD  1000 n

In [ ]:
def get_transactions(address,api_key):
    params = {
        'module': 'account',
        'action': 'txlist',
        'address': address,
        'apikey': api_key
    }

    response = requests.get(ether_url, params=params)
    if response.status_code == 200:
        transactions = response.json().get('result', [])
        return transactions
    else:
        print(f"Failed to fetch transactions for address: {address}")
        return []

address=['0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205']
transactions = get_transactions(address,ether_api_key)
transactions=pd.DataFrame(transactions)


In [ ]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   blockNumber        314 non-null    object
 1   timeStamp          314 non-null    object
 2   hash               314 non-null    object
 3   nonce              314 non-null    object
 4   blockHash          314 non-null    object
 5   transactionIndex   314 non-null    object
 6   from               314 non-null    object
 7   to                 314 non-null    object
 8   value              314 non-null    object
 9   gas                314 non-null    object
 10  gasPrice           314 non-null    object
 11  isError            314 non-null    object
 12  txreceipt_status   314 non-null    object
 13  input              314 non-null    object
 14  contractAddress    314 non-null    object
 15  cumulativeGasUsed  314 non-null    object
 16  gasUsed            314 non-null    object
 1

In [ ]:
transactions

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName
0,11446347,1607885019,0x8d9395fdd44cfa8542c0cfb4046e1f9157f3eac2a1df...,0,0xf8a14fb49814ac030fc1733f9a2e6ee51e0937680411...,135,0xa6bf93783b522eb281de2518047bfaf7bd2f1434,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,32303102740000000000,21000,55000000000,0,1,0x,,6725498,21000,7294112,0x,
1,11452957,1607973411,0xb037552b514d720394b34b4c88dd95d6a6a79847532f...,0,0x842cd4afbc7df1d7f274856c789b9693eb405a4afd6e...,21,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0x00000000219ab540356cbb839cbe05303d7705fa,32000000000000000000,88983,66000000000,0,1,0x22895118000000000000000000000000000000000000...,,1525510,63875,7287502,0x22895118,"deposit(bytes pubkey, bytes withdrawal_credent..."
2,11551169,1609275122,0xbb6e3ef9fb84f9e167008dec26d6a0b8fe9271de1a5c...,1,0x0494942c95b92c6738e9f52ed427300c5cfaade31a01...,65,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,7266478826646914,174581,41000000000,0,1,0xfb3bdb41000000000000000000000000000000000000...,,6939216,133426,7189290,0xfb3bdb41,"swapETHForExactTokens(uint256 amountOut, addre..."
3,11551459,1609279279,0x3528ddda90bcd0f776c1c91ac19e26bb791a0fc71270...,2,0x6c1f7371dfd5a2595487421791d5486cf85b860185e4...,151,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0x881d40237659c251811cec9c364ef91dc08d300c,138100000000000000,336194,47000000000,0,1,0x5f575529000000000000000000000000000000000000...,,11299824,174544,7189000,0x5f575529,"swap(string aggregatorId, address tokenFrom, u..."
4,11569636,1609519357,0x9f50e84215715ac9a7d41faf840b972efd4a95b0cfc1...,3,0x8b71d7e539669aeebfe15e4084b89b2c640d1aa9b60e...,174,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0x88addf731df28f8bf8e71720daee0c441b9c3f73,138520225156004196,21000,65000000000,0,1,0x,,10572189,21000,7170823,0x,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,16670983,1676910971,0x6973151b13ee6f8d59e727ecd767024f21fffc5802f8...,239,0x969e7088fb72d2f57d79885178f027d7df7ea656ed70...,25,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0x3aada3e213abf8529606924d8d1c55cbdc70bf74,0,55930,45252728639,0,1,0x095ea7b3000000000000000000000000ada31f59e70a...,,2187209,46609,2069476,0x095ea7b3,"approve(address _spender, uint256 _value)"
310,16670986,1676911007,0xbb058b69678215eeb9a9a87ef086dc0b7d2f16f217aa...,240,0xb7a61af0345703a3b8afba0c90daf3ffaa62a0eda633...,31,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0xada31f59e70ad18665380f21ce49d4c43f9865c2,0,177926,45515373764,0,1,0x66dfbfb4000000000000000000000000000000000000...,,3226578,142640,2069473,0x66dfbfb4,"lock(uint256 tokenId, address locker)"
311,16671055,1676911847,0xb0b2b8021bc24b41a0497423b7d877cc3d80dc152f20...,241,0xd9d608881a9d9f8618932b7b0e5ac8d86a07c5597612...,155,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0,66836,42210191218,0,1,0x095ea7b3000000000000000000000000ba1222222222...,,13191386,60311,2069404,0x095ea7b3,"approve(address _spender, uint256 _value)"
312,16671060,1676911907,0xff5d0d472fc04e49988f1601378e2c6a55bc32e12f5a...,242,0xa1cc580ccce3a1395eb2657d3e69141cdfc79d082b82...,115,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0xba12222222228d8ba445958a75a0704d566bf2c8,0,37254,44210750531,0,1,0x945bcec9000000000000000000000000000000000000...,,9929747,29122,2069399,0x945bcec9,"batchSwap(uint8 kind, tuple[] swaps, address[]..."


In [ ]:

# Assuming df is the DataFrame you obtained from the previous code
origin_transactions = {}  # Dictionary to store DataFrames for each origin address

for i in range(50):  # Iterate over the first 5 rows
    address = df.loc[i, 'origin']  # Assuming 'origin' is the column name
    transactions = get_transactions(address, ether_api_key)
    transactions_df = pd.DataFrame(transactions)

    # Store the transactions_df in the dictionary with the origin address as the key
    origin_transactions[address] = transactions_df

# Now, origin_transactions is a dictionary where keys are origin addresses and values are DataFrames
# You can access the DataFrames using the respective origin addresses
for address, transactions_df in origin_transactions.items():
    print(f"Transactions for origin {address}:\n{transactions_df}")


Streaming output truncated to the last 5000 lines.
2     0xc098b2a3aa256d2140208c3de6543aaef5cd3a94   
3     0xc098b2a3aa256d2140208c3de6543aaef5cd3a94   
4     0x9a064c50475b81666acdd0e1ce9cead78317b070   
...                                          ...   
2907  0x9a064c50475b81666acdd0e1ce9cead78317b070   
2908  0x9a064c50475b81666acdd0e1ce9cead78317b070   
2909  0x9a064c50475b81666acdd0e1ce9cead78317b070   
2910  0x9a064c50475b81666acdd0e1ce9cead78317b070   
2911  0x9a064c50475b81666acdd0e1ce9cead78317b070   

                                              to               value     gas  \
0     0x9a064c50475b81666acdd0e1ce9cead78317b070   38857964165902079   50000   
1     0x9a064c50475b81666acdd0e1ce9cead78317b070   25594188136276430   50000   
2     0x9a064c50475b81666acdd0e1ce9cead78317b070  141407120000000000   63000   
3     0x9a064c50475b81666acdd0e1ce9cead78317b070  101229280000000000   63000   
4     0x65b28ed75c12d8ce29d892de9f8304a6d2e176a7   90000000000000000  497706   


## Combine the dataframes horizontally. Add a column in the left with the origin wallet address.
